In [4]:
import chromadb

# Configure ChromaDB with local storage path
client = chromadb.PersistentClient(path="./chroma_db")  # Specify the directory for database files

# Create or access a collection
collection = client.get_or_create_collection("docs_collection")

# Dummy data for testing
collection.add(
    documents=["This is a test document"],
    metadatas=[{"source": "test"}],
    ids=["doc1"]
)

# Example: Query the collection
results = collection.query(query_texts=["test"], n_results=1)

print(results)


C:\Users\SHRIRAM\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [10:45<00:00, 129kiB/s] 


{'ids': [['doc1']], 'embeddings': None, 'documents': [['This is a test document']], 'uris': None, 'data': None, 'metadatas': [[{'source': 'test'}]], 'distances': [[0.8545427842875148]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import chromadb



In [ ]:
# Configure ChromaDB with local storage path
client = chromadb.PersistentClient(path="./chroma_db")  # Specify the directory for database files
# Create or access a collection
collection = client.get_or_create_collection("docs_collection")


In [ ]:
# load emvironment variables
load_dotenv()
api_key=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

In [ ]:
def read_files(files):
    texts = []
    for file in files:
        if file.endswith(".pdf"):
            text=get_text_from_pdf(file)
        elif file.endswith(".txt"):
            text=get_text_from_txt(file)
        else:
            print("File type not supported")
            continue
        texts.append(text)
    return texts

In [ ]:
def get_text_from_pdf(file_path):
    pdf = PdfReader(file_path)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [ ]:
def get_text_from_txt(file_path):
    """
    Reads text from a TXT file.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [ ]:
def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
def get_vectors(chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    # convert embeddings to vectors and store it

In [ ]:
def get_converstaional_chain():
    prompt_template = """
    Pre-Prompt\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [ ]:
def ask_llm(user_prompt):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    # load chroma db with embeddings
    # gets docs after similarity search
    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)


In [ ]:
def main():
    pass